# Análise de Sentimentos em Resenhas de  aplicativos de Streamming


![image.png](images/amz_vs_ntf.png)


## Introdução

Análise de Sentimentos é um estudo computacional que envolve [Mineração de texto](https://pt.wikipedia.org/wiki/Minera%C3%A7%C3%A3o_de_texto) e [Processamento de Linguagem Natural](https://pt.wikipedia.org/wiki/Processamento_de_linguagem_natural).

O principal objetivo da Análise de Sentimentos é extrair informações úteis referentes a sentimentos em textos, informações tais como conhecimentos, críticas e opniões sobre determinado tema.

Podemos usar a Análise de Sentimentos para classificar a polaridade de um texto \(positiva ou negativa\). Na classificação de polaridade, é possível, à partir de um dado texto, categorizá-lo de acordo com o sentimento que  o  mesmo expressa, por meio de algoritmos de **aprendizagem de máquina** com categoria supervisionada \(classificadores\).

Entre os classificadores mais utilizados para essa tarefa, podemos citar o [Naive Bayes](https://www.organicadigital.com/blog/algoritmo-de-classificacao-naive-bayes/) e o [Suport Vector Machine](https://pt.wikipedia.org/wiki/M%C3%A1quina_de_vetores_de_suporte)



## Objetivos
Nosso objetivo é criar modelos para classificar o sentimento dos textos das resenhas dos aplicativos de streaming: **Amazon Prime Video** e **Netflix**. Vamos utilizar as resenhas tanto da Play Store \(Android\) quanto da App Store \(iOS\).

Abaixo temos alguns exemplos de resenhas que irão compor nossa base de dados:

> ![image.png](images/resenha_ios.png) ![image.png](images/resenha_android.png)






## Etapas

Realizamos as seguintes tarefas para alcançarmos o objetivos:

1. Coleta de Dados

2. Pré-Processamento dos Dados

3. Treinamento do Modelo

4. Avaliação do Modelo

### Importando as bibliotecas necessárias

In [2]:
pip install -U gensim

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install python-Levenshtein

  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp38-cp38-win_amd64.whl size=82999 sha256=54a3456db58e1b674a1f4ce7e6f3e4c0ff12a46cc80359a063327f6b0b960c6d
  Stored in directory: c:\users\pichau\appdata\local\pip\cache\wheels\d7\0c\76\042b46eb0df65c3ccd0338f791210c55ab79d209bcc269e2c7
Successfully built python-Levenshtein
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('rslp')
nltk.download('gutenberg')
nltk.download('punkt')
from unicodedata import normalize
from nltk.probability import FreqDist
from gensim import corpora

import re
from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PICHAU\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\PICHAU\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\PICHAU\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PICHAU\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import os
import io
import requests
from pygments.lexers.csound import newline

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import *

Montando um gerenciador de arquivos a partir do seu Google Driver

# Coleta dos dados

As resenhas foram exportadas a partir do site "" - esqueci

# Lendo os dataframes das bases consolidadas - seguir daqui

In [4]:
df_netflix_full_concat = pd.read_csv('df_netflix_concat.csv',
                                     encoding='utf-8')
df_netflix_full_concat['Review'][9436]

'Voltaaaaaaa Naruto dubladooooooooooooooo !!!!!!!!?'

In [5]:
df_amazon_full_concat = pd.read_csv('df_amazon_concat.csv',
                                     encoding='utf-8')
df_amazon_full_concat['Review'][2]

'Precisa evoluir, infelizmente, é comum o travamento das legendas, em especial, no app da smartv, além de alguns conteúdos não terem a opção de dublagem na nossa língua nativa (pt-br). O Menu pode ser mais intuitivo e a interface pode ser melhor trabalhada. Ponto positivo é a edição de cores e tamanhos de legenda, filmografia e link com o IMDB..'

# Funções de Pré-processamento do conteudo de texto das resenhas


In [7]:
# -*- coding: utf-8 -*-


def remover_stopwords(texto):
  stop_words = set(stopwords.words('portuguese'))
  stop_words_us = set(stopwords.words('english'))
  #new_stopwords = obter_lista_novas_stopwords(caminho_stopwords)
  #stop_words.update(new_stopwords)
  #stop_words.update([stp.lower() for stp in new_stopwords])
  stop_words.update(stop_words_us)
  palavras_uteis = [palavra for palavra in texto.lower().split() if palavra not in stop_words]
  return ' '.join(palavras_uteis)

def tokenizar_texto(texto):
  texto_tokenizado = word_tokenize(texto.lower())
  return texto_tokenizado

def remover_acentos(texto):
  return normalize('NFKD', texto).encode('ASCII', 'ignore').decode('ASCII')

def converter_texto_para_minusculo(texto):
  return texto.lower()

def remover_caracteres_especiais(texto):
  texto_sem_carac_esp = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕàÀìÌòÒïÏñÑÜüçÇ ]', "", texto)
  return texto_sem_carac_esp

def remover_email(texto):
  padrao_email = r'[\w+\./{1}]+@(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}'
  texto_sem_email = re.sub(padrao_email, "", texto)
  return texto_sem_email

def remover_conjunto_emails(texto):
  padrao_email = r'\{[\w+\./{1}\,*]+\}@(?:[a-zA-Z0-9-]+\.)+[a-zA-Z]{2,}'
  texto_sem_email = re.sub(padrao_email, "", texto)
  return texto_sem_email

def remover_enderecos_emails(texto):
  texto = remover_conjunto_emails(texto)
  texto = remover_email(texto)
  return texto

def remover_url(texto):
  padrao_url = r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*'
  texto_sem_url = re.sub(padrao_url, "", str(texto))
  return texto_sem_url


def remover_numeros(texto):
  texto_sem_numeros = re.sub(r'\d', "", texto)
  return texto_sem_numeros

def remover_caracteres_especiais(texto):
  texto_sem_carac_esp = re.sub(r'[^a-zA-Z0-9áéíóúÁÉÍÓÚâêîôÂÊÎÔãõÃÕàÀÜüçÇ ]', "", texto)
  return texto_sem_carac_esp

def remover_sequencia_espaco_branco(texto):
  padrao = re.compile(r"(\s)\1{1,}")
  return padrao.sub(r"\1", texto)

def remover_palavras_por_tamanho(texto, limite):
  lista_texto = tokenizar_texto(texto)
  lista_palavras_retorno = []
  separador = ' '
  for palavra in lista_texto:
      if len(palavra) > limite:
          lista_palavras_retorno.append(palavra)
  texto_pos_remocao = separador.join(lista_palavras_retorno)
  return texto_pos_remocao

def unificar_artigos_com_n_gramas(lista_de_lista_de_texto):
  artigo_tokens_unificados = []
  for texto in lista_de_lista_de_texto:
      xtexto = texto
      unigramas = xtexto
      xbigramas = construir_bigramas(xtexto)
      xtrigramas = construir_trigramas(xtexto)
      xunigramas = unigramas + xbigramas + xtrigramas
      artigo_tokens_unificados.append(xunigramas)
  return artigo_tokens_unificados

def construir_bigramas(texto_tokenizado):
  bigrams = []
  for i in range(0, len(texto_tokenizado)):
      if (i == len(texto_tokenizado) - 1):
          break
      else:
          bigrama_obs = texto_tokenizado[i] + '_' + texto_tokenizado[i + 1]
          bigrams.append(bigrama_obs)
  # colocar analise por frequencia aqui
  bigrams = remover_ngram_por_frequencia(bigrams, 7)
  return bigrams

def construir_trigramas(texto_tokenizado):
  trigrams = []
  for i in range(0, len(texto_tokenizado)):
      if (i == len(texto_tokenizado) - 2):
          break
      else:
          trigrama_obs = texto_tokenizado[i] + '_' + texto_tokenizado[i + 1] + '_' + texto_tokenizado[i + 2]
          trigrams.append(trigrama_obs)
  # colocar analise por frequencia aqui
  trigrams = remover_ngram_por_frequencia(trigrams, 7)
  return trigrams

def remover_ngram_por_frequencia(lista_ngrams, frequencia_limiar):
  fdist = FreqDist(lista_ngrams)
  lista_ngrams_retorno = [w for w in lista_ngrams if fdist[w] >= frequencia_limiar]
  return lista_ngrams_retorno

def obter_dicionario(lista_de_lista_textos):
  dic_id_palavras = corpora.Dictionary(lista_de_lista_textos)
  return dic_id_palavras

def obter_corpus(dicionario_texto, lista_de_lista_texto):
  corpus_texto = [dicionario_texto.doc2bow(lista_texto) for lista_texto in lista_de_lista_texto]
  return corpus_texto


def remover_hashtags(text):
  raw = re.sub(r'#[A-Za-z0-9]+', "", text)
  return raw

def remover_repeticao_letras(text):
  pattern = re.compile(r"(.)\1{2,}")
  return pattern.sub(r"\1", text)

def traduzir_emoticons(text):
  text = re.sub(r'\:\)|\(\:|\=\)|\(\=|\:D', "sorrindo", text)
  text = re.sub(r'D\:', "surpreso", text)
  text = re.sub(r'\;\)|\(\;', "piscando", text)
  text = re.sub(r'xd', "sorrindo", text)
  text = re.sub(r'\:O', "surpreso", text)
  text = re.sub(r'<2|<3|s2|sz|s3', "amor", text)
  text = re.sub(r'u.u', "prevalecido", text)
  text = re.sub(r'\:\/', "indeciso", text)
  text = re.sub(r'\:\'\(', "chorando", text)
  text = re.sub(r'\:9', "gostando", text)
  text = re.sub(r'\:x', "aborrecido", text)
  text = re.sub(r'\*\-\*', "gostando", text)
  return text

def tradutor_abreveacao(text):
  text = re.sub(r'blz', "beleza", text)
  text = re.sub(r'flw', "tchau", text)
  text = re.sub(r'vlw', "obrigado", text)
  text = re.sub(r'ta', "esta", text)
  text = re.sub(r'mt', "muito", text)
  text = re.sub(r'mts', "muitos", text)
  text = re.sub(r'mó', "muito", text)
  text = re.sub(r'pf', "por favor", text)
  text = re.sub(r'mlhr', "melhor", text)
  text = re.sub(r'q', "que", text)
  text = re.sub(r'n', "não", text)
  text = re.sub(r's', "sim", text)
  text = re.sub(r'pq', "porque", text)
  text = re.sub(r'ok', "beleza", text)
  text = re.sub(r'vcs', "voces", text)
  text = re.sub(r'vc', "voce", text)
  text = re.sub(r'amr', "amor", text)
  text = re.sub(r'migo', "amigo", text)
  text = re.sub(r'migs', "amigo", text)
  text = re.sub(r'okz', "beleza", text)
  text = re.sub(r'hj', "hoje", text)
  text = re.sub(r'p|p\/', "para", text)
  return text

def tokenizar_tweets(text):
  tweet_tokenizer =  TweetTokenizer(strip_handles=True, reduce_len=False, preserve_case=False)
  return tweet_tokenizer.tokenize(text)

def converter_termos_geral(texto):
  texto = re.sub(" app"," aplicativo ",texto)
  texto = re.sub('blz ', " beleza ", texto)
  texto = re.sub('ta ', " esta ", texto)
  texto = re.sub('mt', " muito ", texto)
  texto = re.sub('mts', " muitos ", texto)
  texto = re.sub('mó ', " muito ", texto)
  texto = re.sub(' pf ', " por favor ", texto)
  texto = re.sub(' mlhr ', " melhor ", texto)
  texto = re.sub(' q ', " que ", texto)
  texto = re.sub(' n ', " não ", texto)
  texto = re.sub(' s ', " sim ", texto)
  texto = re.sub(' pq ', " porque ", texto)
  texto = re.sub(' vcs ', " voces ", texto)
  texto = re.sub(' vc ', " voce ", texto)
  texto = re.sub(' amr ', " amor ", texto)
  texto = re.sub(' hj ', " hoje ", texto)
  texto = texto.lstrip()
  return texto.rstrip()


def pre_processamento_dados(texto):
  texto = str(texto)
  texto = remover_url(texto)
  texto = remover_hashtags(texto)
  texto = remover_numeros(texto)
  texto = converter_termos_geral(texto)
  texto = traduzir_emoticons(texto)
  texto = remover_palavras_por_tamanho(texto, 1)
  texto = remover_stopwords(texto)
  #texto = tokenizar_tweets(texto)
  texto = remover_repeticao_letras(texto)
  texto = remover_acentos(texto)
  texto = remover_caracteres_especiais(texto)
  texto = remover_sequencia_espaco_branco(texto)
  texto = remover_hashtags(texto)
  return texto

# Preparando o corpus para o treinamento

In [ ]:
df_amazon_full_concat['Classification'] = 'NaN'
df_netflix_full_concat['Classification'] = 'NaN'

In [ ]:
df_amazon_full_concat.head()


In [ ]:
df_netflix_full_concat.head()

In [ ]:
print("Df netflix antes de remover linhas duplicas: {}".format(len(df_netflix_full_concat)))
print("Df amazon antes de remover linhas duplicas: {}".format(len(df_amazon_full_concat)))

In [ ]:
# Removendo os valores duplicados:
df_netflix_full_concat.drop_duplicates(['Review'], inplace=True)
df_amazon_full_concat.drop_duplicates(['Review'], inplace=True)

In [ ]:
print("Df netflix depois de remover linhas duplicas: {}".format(len(df_netflix_full_concat)))
print("Df amazon depois de remover linhas duplicas: {}".format(len(df_amazon_full_concat)))

### Classificando a coluna de sentimento (Classificação)

As avaliações das resenhas nas lojas de Apps são realizadas através de estrelas,  que  equivalem  a  notas, podendo ser dadas de 1 (pior avaliação) até 5 (melhor avaliação). Através dessa avaliação foi feita a anotação da polaridade dos sentimentos contidos nas resenhas:


* 4 ou 5 estrelas foram classificadas como positivas;



* 1 ou 2 estrelas foram consideradas negativas. 




Avaliações neutras (3 estrelas) foram removidas.

In [ ]:
for index, row in df_amazon_full_concat.iterrows():
  if str(row['Rating']) in ['1', '2']:
    df_amazon_full_concat.loc[index,'Classification'] = "Negativa"
  if str(row['Rating']) in ['4', '5']:
    df_amazon_full_concat.loc[index,'Classification'] = "Positiva"

In [ ]:
for index, row in df_netflix_full_concat.iterrows():
  if str(row['Rating']) in ['1', '2']:
    df_netflix_full_concat.loc[index,'Classification'] = "Negativa"
  if str(row['Rating']) in ['4', '5']:
    df_netflix_full_concat.loc[index,'Classification'] = "Positiva"

In [ ]:
df_amazon_full_concat.Classification

In [ ]:
df_netflix_full_concat.Classification

In [ ]:
df_amazon_full_concat.Classification.value_counts().plot(kind='bar', )

In [ ]:
df_netflix_full_concat.Classification.value_counts().plot(kind='bar')

### Separando a base classificada e não classificada
- Elas serão usadas no treinamento do modelo e no teste do modelo
- Ponto de discussão: Será que faz sentido adotar a classificação automática? 
Quantas resenhas não-classificadas nós temos?

In [ ]:
df_amazon_rotulado =  df_amazon_full_concat.query('Classification != "NaN"')
df_amazon_nao_rotulado =  df_amazon_full_concat.query('Classification == "NaN"')
print("Tamanho do Dataframe Amazon Rotulado: {0}".format(len(df_amazon_rotulado)))
print("Tamanho do Dataframe Amazon Não Rotulado: {0}".format(len(df_amazon_nao_rotulado)))

In [ ]:
len(df_amazon_nao_rotulado)

In [ ]:
df_netflix_rotulado =  df_netflix_full_concat.query('Classification != "NaN"')
df_netflix_nao_rotulado =  df_netflix_full_concat.query('Classification == "NaN"')
print("Tamanho do Dataframe Netflix Rotulado: {0}".format(len(df_netflix_rotulado)))
print("Tamanho do Dataframe Netflix Não Rotulado: {0}".format(len(df_netflix_nao_rotulado)))

In [ ]:
len(df_netflix_nao_rotulado)

### Separando as resenhas das classes

In [ ]:
resenhas_amazon = df_amazon_rotulado['Review']
classes_amazon = df_amazon_rotulado['Classification']


In [ ]:
resenhas_netflix = df_netflix_rotulado['Review']
classes_netflix = df_netflix_rotulado['Classification']


### Realizando o pre-processamento nas resenhas

In [ ]:
resenhas_lista_amazon = [pre_processamento_dados(i).lstrip().rstrip() for i in resenhas_amazon]


In [ ]:
resenhas_lista_netflix = [pre_processamento_dados(i).lstrip().rstrip() for i in resenhas_netflix]

In [ ]:
resenhas_lista_amazon[:10]

In [ ]:
resenhas_lista_netflix[:10]

In [ ]:
resenhas_vazias= 0
for res in resenhas_lista:
  if len(res) ==0:
    resenhas_vazias = resenhas_vazias +1
percent_resenhas_vazias = resenhas_vazias/len(resenhas_lista)
print("Resenhas Vazias: {0}%".format(round(percent_resenhas_vazias*100, 2)))

# Criando o modelo

### vetorização dos textos

In [ ]:
tweet_tokenizer = TweetTokenizer()
vectorizer = CountVectorizer(analyzer="word", tokenizer=tweet_tokenizer.tokenize)

Vetorizando os dados
- cada palavra se torna uma coluna - matriz de termo-frequencia
- ponto de atenção - Uso do vectorizer.transform() e o vectorizer.fit_transform() 

In [ ]:
freq_resenhas_amazon = vectorizer.fit_transform(resenhas_lista_amazon)
freq_resenhas_netflix = vectorizer.fit_transform(resenhas_lista_netflix)

Dados vetorizados em uma matriz esparsa, de "0" e "1"

In [ ]:
type(freq_resenhas_amazon)

In [ ]:
freq_resenhas_amazon.A

Tamanho das matrizes esparsas de Termo-Frequencia criadas após a vetorização

In [ ]:
tp_matrix_am = freq_resenhas_amazon.shape
tp_matrix_nt = freq_resenhas_netflix.shape

print("Dimensões da Matriz Esparsa do Df Amazon: {0} linhas x {1} colunas".format(tp_matrix_am[1], tp_matrix_am[0]))
print("Dimensões da Matriz Esparsa do Df Netflix: {0} linhas x {1} colunas".format(tp_matrix_nt[1], tp_matrix_nt[0]))


### Treino do Modelo

In [ ]:
modelo_amazon = MultinomialNB()
modelo_amazon.fit(freq_resenhas_amazon, classes_amazon)

In [ ]:
modelo_netflix = MultinomialNB()
modelo_netflix.fit(freq_resenhas_netflix, classes_netflix)

## Verificando o modelo com uma frase de teste

In [8]:
teste = ['possui excelentes series precisa melhorar a busca',
         'estou muito feliz com o aplicativo',
         'não estou muito feliz com o aplicativo',
         'aplicativo não abre e lento']

In [ ]:
teste = [pre_processamento_dados(i).lstrip().rstrip() for i in teste]

In [ ]:
freq_teste = vectorizer.transform(teste)

Modelo Netflix

In [ ]:
for t,c in zip(teste, modelo_netflix.predict(freq_teste)):
  print(t + ", " + c)

In [ ]:
# Detalhando a prob. da predição
modelo_netflix.predict_proba(freq_teste).round(2)

Modelo Amazon

In [ ]:
for t,c in zip(teste, modelo_amazon.predict(freq_teste)):
  print(t + ", " + c)

In [ ]:
# Detalhando a prob. da predição
modelo_amazon.predict_proba(freq_teste).round(2)

# Definindo um Pipeline automatizado para gerar o modelo

Etapas do Pipiline Simples






In [ ]:
etapas = [('counts', CountVectorizer()),
          ('classifier', MultinomialNB())]


In [ ]:
pipeline_amazon = Pipeline(etapas)
pipeline_netflix = Pipeline(etapas)

Treinando os Pipelines

In [ ]:
pipeline_amazon.fit(resenhas_lista_amazon, classes_amazon)

In [ ]:
pipeline_netflix.fit(resenhas_lista_netflix, classes_netflix)

In [ ]:
pipeline_amazon.steps

Pipeline de Modelo SVM
- Kernel Linear


Etapas do Pipilene SVM

In [ ]:
etapas_svm = [('count', CountVectorizer()),
              ('classifier',svm.SVC(kernel='linear'))]

In [ ]:
pipeline_svm_amazon = Pipeline(etapas_svm)
pipeline_svm_netflix = Pipeline(etapas_svm)

# Validando os modelos
- Validação Cruzada - de 10 folds - cv=10

In [ ]:
resultados_modelo_amazon = cross_val_predict(pipeline_amazon, resenhas_lista_amazon, classes_amazon, cv=10)
resultados_modelo_netflix = cross_val_predict(pipeline_netflix, resenhas_lista_netflix, classes_netflix, cv=10)


* Medindo a métrica de acurácia dos modelos


In [ ]:
accur_pipeline_amazon = accuracy_score(classes_amazon, resultados_modelo_amazon)
print("Acurácia do Modelo Pipeline Simples Amazon: {0}".format((accur_pipeline_amazon)))

accur_pipeline_netflix = accuracy_score(classes_netflix, resultados_modelo_netflix)
print("Acurácia do Modelo Pipeline Simples Netflix: {0}".format((accur_pipeline_netflix)))

Medidas de Validação do Modelo

* Acurácia: proximidade entre o valor obtido no modelo e o valor verdadeiro na classificação rotulada 





Amazon

In [ ]:
# Acurácia
sentimentos = ['Positiva', 'Negativa']
print(classification_report(classes_amazon, resultados_modelo_amazon, sentimentos))

Netflix

In [ ]:
# Acurácia
sentimentos = ['Positiva', 'Negativa']
print(classification_report(classes_netflix, resultados_modelo_netflix, sentimentos))

* Matriz de Confusão: Avaliar no número de predições erradas por classe (feature)

Amazon


In [ ]:
print(pd.crosstab(classes_amazon, resultados_modelo_amazon, rownames=['Real'], colnames=['Predito'],  margins=True))

In [ ]:
print(pd.crosstab(classes_netflix, resultados_modelo_netflix, rownames=['Real'], colnames=['Predito'],  margins=True))

In [ ]:
def obter_metricas(modelo, resenhas, classes):
  resutado = cross_val_predict(modelo, resenhas, classes,cv=10)
  return accuracy_score(classes, resutado)
  

In [ ]:
# naive bayes simples
accur_pp_amazon = obter_metricas(pipeline_amazon, resenhas_lista_amazon, classes_amazon)
accur_pp_netflix = obter_metricas(pipeline_netflix, resenhas_lista_netflix, classes_netflix)

print("Acurácia do modelo Naive Bayes - Amazon: {}".format(accur_pp_amazon))
print("Acurácia do modelo Naive Bayes - Netflix {}".format(accur_pp_netflix))

In [ ]:
# SVM
accur_pp_svm_amazon = obter_metricas(pipeline_svm_amazon, resenhas_lista_amazon, classes_amazon)
accur_pp_svm_netflix = obter_metricas(pipeline_svm_netflix, resenhas_lista_netflix, classes_netflix)

print("Acurácia do modelo SVM - Amazon: {}".format(accur_pp_svm_amazon))
print("Acurácia do modelo SVM - Netflix: {}".format(accur_pp_svm_netflix))


# Predição aplicada na Base de Dados
- Vamos classificar as resenhas não-Classificadas 
- Para separar as bases (Positiva e Negativa) que usaremos na Modelagem de Tópicos

---
 ToDo List
* Criar metodo para pre-processar uma string  e retornar string pre-processada
* Criar método para predizer as resenhas não-classificadas diretamente nos Dfs
* Concatenar e depois consolidar os Dfs classificados e nao-classificados
* Separar resenhas classificadas entre Positivas e Negativas
* Consolidar resenhas separadas em arquivo .txt




In [ ]:
# ToDo List
# Criar método para predizer as resenhas não-classificadas diretamente nos Dfs
# Separar resenhas classificadas entre Positivas e Negativas
# Consolidar resenhas separadas em arquivo .txt

In [ ]:
df_net_nao_rotulado.head()

In [ ]:
df_netflix_nao_rotulado.head()

Classificando resenhas não-rotuladas com o modelo Netflix

In [ ]:
count = 0
for index, row in df_netflix_nao_rotulado.iterrows():
  resenha = row['Review']
  resenha = pre_processamento_dados(resenha)
  if (not(resenha.isspace()) and len(resenha) > 1):
    lista_resenha = []
    lista_resenha.append(resenha)
    classificacao_pred = pipeline_netflix.predict(lista_resenha)[0]
    df_netflix_nao_rotulado.loc[index,'Classification'] = classificacao_pred
    #print("{0},{1},{2}".format(index, resenha,  classificacao_pred))
    count = count+1


print(count)

In [ ]:
len(df_netflix_nao_rotulado)

Classificando resenhas não-rotuladas com o modelo Amazon

In [ ]:
count = 0
for index, row in df_amazon_nao_rotulado.iterrows():
  resenha = row['Review']
  resenha = pre_processamento_dados(resenha)
  if (not(resenha.isspace()) and len(resenha) > 1):
    lista_resenha = []
    lista_resenha.append(resenha)
    classificacao_pred = pipeline_amazon.predict(lista_resenha)[0]
    df_amazon_nao_rotulado.loc[index,'Classification'] = classificacao_pred
    #print("{0},{1},{2}".format(index, resenha,  classificacao_pred))
    count = count+1


print(count)

In [ ]:
len(df_amazon_nao_rotulado)

In [ ]:
ls

In [ ]:
df_amazon_nao_rotulado.head()

In [ ]:
df_netflix_nao_rotulado.head()

In [ ]:
df_amazon_nao_rotulado.to_csv("df_amazon_rotulada_modelo.csv") 
df_netflix_nao_rotulado.to_csv("df_netflix_rotulada_modelo.csv") 